## 基于经典网络架构训练图像分类模型

### 数据预处理
- 数据增强：使用torchvision中自带的transform模块自带的功能
- 数据预处理：使用torchvision中自带的datasets模块自带的功能
- Dataloader模块直接读取batch数据


### 网络模块设置：
- 加载预训练模型，torchvision中有很多经典的网络架构，调用起来也是十分方便的，并且可以用别人训练好的权重参数来继续训练，也就是所谓的迁移学习
- 需要注意的是别人训练好的任务可能与我们的任务不是我安全一样的，需要吧最后的head层改一改，一般也就是最后的全连接层，改成咋嫩自己的任务
- 训练的时候可以全部从头训练，也可以只训练咱们最后的层。因为前几层都是做特征提取的，本事任务目标是一致的，所以可以直接用别人训练好的参数，只训练最后的层

### 网络模型保存预测试
- 模型保存的时候可以带有选择性， 例如在验证集中如果当前效果好则保存
- 读取模型进行实际测试

In [9]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, models, datasets
import imageio
import time
import random
import sys
import json
from PIL import Image

- torchvision中内置了很多的经典网络架构，可以直接调用

### 下载数据

### 数据读取和预处理操作

### 制作好数据源：
- data_transforms中指定了所有图像预处理操作
- imageFolder假设所有的文件按文件夹保存好，每个文件下面是同一类的文件，文件夹的名字为他们分类的名字

### 数据预处理
官方提供的数据集需要处理成目录形式如下
flower_data
-- train
  --class1
  --class2
  ...
-- test
-- valid

In [10]:
import scipy.io
import numpy as np
import os
from PIL import Image
import shutil


# 获取lable
labels = scipy.io.loadmat('../data/102flowers/imagelabels.mat')  # 该地址为imagelabels.mat的相对地址
labels = np.array(labels['labels'][0]) - 1
# print("labels:", labels)

# 分离train、test、valid
setid = scipy.io.loadmat('../data/102flowers/setid.mat')  # 该地址为setid.mat的相对地址

validation = np.array(setid['valid'][0]) - 1
np.random.shuffle(validation)

train = np.array(setid['trnid'][0]) - 1
np.random.shuffle(train)

test = np.array(setid['tstid'][0]) - 1
np.random.shuffle(test)

# 把数据导入flower_dir
flower_dir = list()

for img in os.listdir("../data/102flowers/jpg"):  # 该地址为源数据图片的相对地址
    flower_dir.append(os.path.join("../data/102flowers/jpg", img))

flower_dir.sort()
# print(flower_dir[0])

In [11]:
# train_dir
path_train = "../data/102flowers/train"
os.makedirs(path_train, exist_ok=True)
des_folder_train = path_train  # 该地址可为新建的训练数据集文件夹的相对地址
for tid in test:
    #打开图片并获取标签
    img = Image.open(flower_dir[tid])
    # print(img)
    # print(flower_dir[tid])
    # img = img.resize((256, 256), Image.ANTIALIAS)
    lable = labels[tid]
    # print(lable)
    path = flower_dir[tid]
    # print("path:", path)
    base_path = os.path.basename(path)
    # print("base_path:", base_path)
    classes = "c" + str(lable)
    class_path = os.path.join(des_folder_train, classes)
    # 判断结果
    if not os.path.exists(class_path):
        os.makedirs(class_path)
    # print("class_path:", class_path)
    despath = os.path.join(class_path, base_path)
    # print("despath:", despath)
    img.save(despath)

In [12]:
# valid_dir
path_valid = "../data/102flowers/valid"
os.makedirs(path_valid, exist_ok=True)
des_folder_validation = path_valid  # 该地址可为新建的训练数据集文件夹的相对地址

for tid in validation:
    img = Image.open(flower_dir[tid])
    # print(flower_dir[tid])
    # img = img.resize((256, 256), Image.ANTIALIAS)
    lable = labels[tid]
    # print(lable)
    path = flower_dir[tid]
    # print("path:", path)

    base_path = os.path.basename(path)
    # print("base_path:", base_path)
    classes = "c" + str(lable)
    class_path = os.path.join(des_folder_validation, classes)
    # 判断结果
    if not os.path.exists(class_path):
        os.makedirs(class_path)
    # print("class_path:", class_path)
    despath = os.path.join(class_path, base_path)
    # print("despath:", despath)
    img.save(despath)



In [13]:
# test_dir
path_test = "../data/102flowers/test"
os.makedirs(path_test, exist_ok=True)
des_folder_test = path_test # 该地址可为新建的训练数据集文件夹的相对地址

for tid in train:
    img = Image.open(flower_dir[tid])
    # print(flower_dir[tid])
    # img = img.resize((256, 256), Image.ANTIALIAS)
    lable = labels[tid]
    # print(lable)
    path = flower_dir[tid]
    # print("path:", path)
    base_path = os.path.basename(path)
    # print("base_path:", base_path)
    classes = "c" + str(lable)
    class_path = os.path.join(des_folder_test, classes)
    # 判断结果
    if not os.path.exists(class_path):
        os.makedirs(class_path)
    # print("class_path:", class_path)
    despath = os.path.join(class_path, base_path)
    # print("despath:", despath)
    img.save(despath)



In [14]:
data_transforms = {
    'train':
        transforms.Compose(
            [
                transforms.Resize([96,96]),              # 缩放图片为96*96的大小
                transforms.RandomRotation(45),           # 随机旋转45度(-45,45)之间随机选
                transforms.CenterCrop(64),               # 从中心裁剪64*64的图片，64是组中图片的大小
                transforms.RandomHorizontalFlip(0.5),    # 随机水平翻转
                transforms.RandomVerticalFlip(0.5),      # 随机垂直翻转
                transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
                                                         # 随机改变亮度、对比度、饱和度和色调
                transforms.RandomGrayscale(p=0.025),     # 随机将图片转为灰度图
                transforms.ToTensor(),                   # 将图片转为Tensor
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 标准化
            ]
        ),
    'valid':
        transforms.Compose(
            [
                transforms.Resize([64,64]),                # 缩放图片为96*96的大小
                transforms.ToTensor(),                   # 将图片转为Tensor
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                         # 标准化要和训练集的一样
                # 验证集合是否需要做数据增强，取决于训练好模型之后实际的使用场景中是什么情况。
                # 例如。如果预测的环境曝光值比较高，就需要我们对验证集做一些数据增强，比如亮度、对比度、饱和度等
                # 再比如，如果预测的环境中有很多噪声，就需要我们对验证集做一些数据增强，比如高斯噪声、椒盐噪声等
                # 再比如，如果预测的环境中只能看到一部分图像，就需要我们对验证集做一些数据增强，比如裁剪、旋转等
            ]
        )
}

# 构建dataloader， dataset


In [15]:
from torchvision.datasets import ImageFolder
data_dir = '../data/102flowers/'

image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128, shuffle=True,) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [16]:
print(len(class_names))
print(class_names)

102
['c0', 'c1', 'c10', 'c100', 'c101', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c2', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c3', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c4', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c5', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c6', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c7', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c8', 'c80', 'c81', 'c82', 'c83', 'c84', 'c85', 'c86', 'c87', 'c88', 'c89', 'c9', 'c90', 'c91', 'c92', 'c93', 'c94', 'c95', 'c96', 'c97', 'c98', 'c99']


-trnid字段:总共有1020列，每10列为一类花卉的图片，每列上的数字代表图片号。
-valid字段:总共有1020列，每10列为一类花卉的图片，每列上的数字代表图片号。
-tstid字段:总共有6149列，每一类花卉的列数不定，每列上的数字代表图片号。

In [17]:
print(dataset_sizes)

{'train': 6149, 'valid': 1020}


In [18]:

with open('../data/102flowers/cat_to_name.json','r') as f:
    cat_to_name = json.load(f)
cat_to_name

{'21': 'fire lily',
 '3': 'canterbury bells',
 '45': 'bolero deep blue',
 '1': 'pink primrose',
 '34': 'mexican aster',
 '27': 'prince of wales feathers',
 '7': 'moon orchid',
 '16': 'globe-flower',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '79': 'toad lily',
 '39': 'siam tulip',
 '24': 'red ginger',
 '67': 'spring crocus',
 '35': 'alpine sea holly',
 '32': 'garden phlox',
 '10': 'globe thistle',
 '6': 'tiger lily',
 '93': 'ball moss',
 '33': 'love in the mist',
 '9': 'monkshood',
 '102': 'blackberry lily',
 '14': 'spear thistle',
 '19': 'balloon flower',
 '100': 'blanket flower',
 '13': 'king protea',
 '49': 'oxeye daisy',
 '15': 'yellow iris',
 '61': 'cautleya spicata',
 '31': 'carnation',
 '64': 'silverbush',
 '68': 'bearded iris',
 '63': 'black-eyed susan',
 '69': 'windflower',
 '62': 'japanese anemone',
 '20': 'giant white arum lily',
 '38': 'great masterwort',
 '4': 'sweet pea',
 '86': 'tree mallow',
 '101': 'trumpet creeper',
 '42': 'daffodil',
 '22': 'pincushion flower',
 

加载torch提供的预训练模型，这里使用的是resnet18，如果想使用其他的模型，可以在torchvision.models中查看，然后将下面的代码中的resnet18替换为其他模型即可。
- 第一次执行会下载模型的文件

In [19]:
train_on_gpu = torch.cuda.is_available()
if train_on_gpu==False:
    device = torch.device("cpu")
    print('CUDA is not available.  Training on CPU ...')
else:
    device = torch.device("cuda")
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


# 构建模型
## 模型参数要不要更新
- 有时候用人家的模型就一直用了，

In [20]:
# 先冻住全部层
def freeze_parameter_requires_grad(model):
        for param in model.parameters(): # 遍历每一层，设置为反向传播不更新梯度
            param.requires_grad = False

# 把模型的输出层改成自己的，微调预训练模型，Linear默认是更新梯度的
def initialize_model(
        model_name,
        num_classes,
        feature_extract,
        use_pretrained=True):
    if model_name == 'resnet':
        model_ft = models.resnet18(pretrained=use_pretrained)
    if feature_extract:
        freeze_parameter_requires_grad(model_ft)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes) # 重新修改最后一层的全连接层

    input_size = 64
    return model_ft, input_size

# 构建模型
model_name = "resnet"
use_pretrained=True
feature_extract = True

model_ft, input_size = initialize_model(model_name, len(class_names), feature_extract, use_pretrained)

# 模型加载到什么卡上面计算
model_ft = model_ft.to(device)

# 模型保存
filename = 'model.pth'

# 是否训练所有层
param_to_update = model_ft.parameters()  #把模型中所有层名字，权重保存下来


print("Prameters to learn:")
if feature_extract:
    param_to_update = []
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            param_to_update.append(param)
            print("\t", name)
else:
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t", name)


/Users/linweicheng/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/linweicheng/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Prameters to learn:
	 fc.weight
	 fc.bias


# 构建优化器

In [21]:
optimizer_ft = optim.Adam(param_to_update, lr=0.001)
# 衰减策略
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
criterion = nn.CrossEntropyLoss() # 损失函数选择交叉熵

# 训练模块


In [25]:
import copy
from tqdm import tqdm
def train_model(
        model,
        dataloaders,
        criterion,
        optimizer,
        num_epochs=25,
        filename='model.pth'
):
    start = time.time()

    best_acc = 0                #每次判断实际迭代结果是否有提升，并替换记录最好的一次

    model.to(device)            #模型加载到什么卡上面计算

    train_acc_history = []      #记录每次迭代的训练集准确率
    val_acc_history = []        #记录每次迭代的验证集准确率
    train_losses = []
    val_losses = []

    # 学习率
    LRs = [(optimizer.param_groups[0]['lr'])]

    # 定一个一个变量保存模型的权重，开始的时候先初始化为当前模型的权重
    best_model_wts = copy.deepcopy(model.state_dict())

    # 开始迭代
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('\033[34m——\033[0m'*50)

        # 训练和验证
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() # 训练模式
                print('training')
            else:
                model.eval() # 验证模式
                print('validating')

            running_loss = 0.0
            running_corrects = 0

            # 迭代数据, loader中已经确定了batch_size
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 清零
                optimizer.zero_grad()
                # 前向传播
                outputs = model(inputs)
                # 计算损失
                loss = criterion(outputs, labels)
                _, pred = torch.max(outputs, 1)

                # 反向传播
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # 计算损失
                running_loss += loss.item() * inputs.size(0) # loss.item()是一个数，inputs.size(0)是一个batch_size
                running_corrects += torch.sum(pred == labels.data) # pred是一个batch_size的向量，labels.data是一个batch_size的向量

            # 计算损失和准确率
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            # 记录每次迭代的训练集准确率和损失
            time_elapsed = time.time() - start # 一个epoch的时间
            print('\033[34m{} Loss:\033[0m {:.4f}||\033[34mAcc:\033[0m {:.4f}||\033[34mTime:\033[0m {:.0f}m {:.0f}s'.format(
                phase, epoch_loss, epoch_acc, time_elapsed // 60, time_elapsed % 60
            ))

            # 记录最好的那次模型，这一个epoch中最好的那个模型
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                state = {
                    'state_dict': model.state_dict(), # 字典的key是层名字，value是层的权重
                    'bst_acc': best_acc, # 记录最佳准确率
                    'optimizer': optimizer.state_dict(), # 优化器的状态
                }
                torch.save(state, filename)  # 保存模型

            if phase == 'valid':
                 val_acc_history.append(epoch_acc)
                 val_losses.append(epoch_loss)

            if phase == 'train':
                 train_acc_history.append(epoch_acc)
                 train_losses.append(epoch_loss)

        print('optimizer learning rate: {}'.format(optimizer.param_groups[0]['lr']))
        LRs.append(optimizer.param_groups[0]['lr'])
        print()
        scheduler.step() # 更新学习率， 累计到制定好的step_size，就会更新学习率，这里第11个epoch会更新

    time_elapsed = time.time()-start
    print('\033[31mTraining complete in\033[0m {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60
    ))
    print('\033[31mBest val Acc:\033[0m {:4f}'.format(best_acc))

    # 加载最好的模型
    model.load_state_dict(best_model_wts)
    return model, train_acc_history, val_acc_history, train_losses, val_losses, LRs

# 执行训练

In [27]:
model_ft, train_acc_history, val_acc_history, train_losses, val_losses, LRs = train_model(
    model_ft,
    dataloaders,
    criterion,
    optimizer_ft,
    num_epochs=25,
    filename=filename
)

Epoch 1/25
————————————————————————————————————————————————————————————————————————————————————————————————————
training


 14%|█▍        | 7/49 [00:17<01:42,  2.43s/it]


KeyboardInterrupt: 

# 解冻模型，第二轮训练

In [24]:
import torch.optim as optim
import torch

# 加载之前训练好的模型
model_ft, input_size = initialize_model(model_name, num_classes=102, feature_extract=False, use_pretrained=True)

for param in model_ft.parameters():
    param.requires_grad = True

# 继续训练所有参数，lr设置的小一点
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# 损失函数
criterion = nn.CrossEntropyLoss()

# 加载之前的权重模型
check_point = torch.load('model.pth')
best_acc = check_point['bst_acc']
model_ft = model_ft.load_state_dict(check_point['state_dict'])





model_ft, train_acc_history2, val_acc_history2, train_losses2, val_losses2, LRs2 = train_model(
    model_ft,
    dataloaders,
    criterion,
    optimizer,
    num_epochs=10,
    filename=filename
)

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([102, 512]) from checkpoint, the shape in current model is torch.Size([1000, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([102]) from checkpoint, the shape in current model is torch.Size([1000]).